In [1]:
import os, sys
import gc
import time
import operator
import datetime
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# other packages
import numpy as np
from PIL import Image
from imageio import imsave
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils import inception_score_torch as ist
from utils import fid_score_torch as fst
from utils.utils import create_logger, RunningStats, save_checkpoint, set_log_dir

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
torch.cuda.manual_seed(1)

# Generator
- Block design
- Cell design
- Network design

In [2]:
# Generator action types
CONV_TYPE = {0: "post", 1: "pre"}
NORM_TYPE = {0: None, 1: "bn", 2: "in"}
UP_TYPE = {0: "bilinear", 1: "nearest", 2: "deconv"}
SHORT_CUT_TYPE = {0: False, 1: True}
SKIP_TYPE = {0: False, 1: True}

# pre-conv block for generator
class PreGenBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_block, ksize=3):
        super(PreGenBlock, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.inn = nn.InstanceNorm2d(in_channels)
        self.up_block = up_block
        self.deconv = nn.ConvTranspose2d(
            in_channels, in_channels, kernel_size=2, stride=2
        )
        self.conv = nn.Conv2d(in_channels, out_channels, ksize, padding=ksize // 2)

    def set_arch(self, up_id, norm_id):
        self.up_type = UP_TYPE[up_id]
        self.norm_type = NORM_TYPE[norm_id]

    def forward(self, x):
        # norm
        if self.norm_type:
            if self.norm_type == "bn":
                h = self.bn(x)
            elif self.norm_type == "in":
                h = self.inn(x)
            else:
                raise NotImplementedError(self.norm_type)
        else:
            h = x

        # activation
        h = nn.ReLU()(h)

        # whether this is a upsample block
        if self.up_block:
            if self.up_type == "deconv":
                h = self.deconv(h)
            else:
                h = F.interpolate(h, scale_factor=2, mode=self.up_type)

        # conv
        out = self.conv(h)
        return out

# post-conv block for generator
class PostGenBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_block, ksize=3):
        super(PostGenBlock, self).__init__()
        self.deconv = nn.ConvTranspose2d(
            in_channels, in_channels, kernel_size=2, stride=2
        )
        self.conv = nn.Conv2d(in_channels, out_channels, ksize, padding=ksize // 2)
        self.bn = nn.BatchNorm2d(out_channels)
        self.inn = nn.InstanceNorm2d(out_channels)
        self.up_block = up_block

    def set_arch(self, up_id, norm_id):
        self.up_type = UP_TYPE[up_id]
        self.norm_type = NORM_TYPE[norm_id]

    def forward(self, x):
        # whether this is a upsample block
        if self.up_block:
            if self.up_type == "deconv":
                h = self.deconv(x)
            else:
                h = F.interpolate(x, scale_factor=2, mode=self.up_type)
        else:
            h = x

        # conv
        h = self.conv(h)

        # norm
        if self.norm_type:
            if self.norm_type == "bn":
                h = self.bn(h)
            elif self.norm_type == "in":
                h = self.inn(h)
            else:
                raise NotImplementedError(self.norm_type)

        # activation
        out = nn.ReLU()(h)

        return out


class GCell(nn.Module):
    def __init__(self, in_channels, out_channels, num_skip_in, ksize=3):
        super(GCell, self).__init__()

        self.post_conv1 = PostGenBlock(in_channels, out_channels, ksize=ksize, up_block=True)
        self.pre_conv1 = PreGenBlock(in_channels, out_channels, ksize=ksize, up_block=True)

        self.post_conv2 = PostGenBlock(out_channels, out_channels, ksize=ksize, up_block=False)
        self.pre_conv2 = PreGenBlock(out_channels, out_channels, ksize=ksize, up_block=False)

        self.deconv_sc = nn.ConvTranspose2d(in_channels, in_channels, kernel_size=2, stride=2)
        self.c_sc = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        # skip_in
        self.skip_deconvx2 = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.skip_deconvx4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2),
        )

        self.num_skip_in = num_skip_in
        if num_skip_in:
            self.skip_in_ops = nn.ModuleList([
                    nn.Conv2d(in_channels, out_channels, kernel_size=1)
                    for _ in range(num_skip_in)   
            ])

    def set_arch(self, conv_id, norm_id, up_id, short_cut_id, skip_ins):
        self.post_conv1.set_arch(up_id, norm_id)
        self.pre_conv1.set_arch(up_id, norm_id)
        self.post_conv2.set_arch(up_id, norm_id)
        self.pre_conv2.set_arch(up_id, norm_id)

        if self.num_skip_in:
            self.skip_ins = [0 for _ in range(self.num_skip_in)]
            for skip_idx, skip_in in enumerate(decimal2binary(skip_ins)[::-1]):
                self.skip_ins[-(skip_idx + 1)] = int(skip_in)

        self.conv_type = CONV_TYPE[conv_id]
        self.up_type = UP_TYPE[up_id]
        self.short_cut = SHORT_CUT_TYPE[short_cut_id]

    def forward(self, x, skip_ft=None):
        residual = x

        # first conv
        if self.conv_type == "post":
            h = self.post_conv1(residual)
        elif self.conv_type == "pre":
            h = self.pre_conv1(residual)
        else:
            raise NotImplementedError(self.norm_type)
        _, _, ht, wt = h.size()
        h_skip_out = h
        # second conv
        if self.num_skip_in:
            assert len(self.skip_in_ops) == len(self.skip_ins)
            for skip_flag, ft, skip_in_op in zip(self.skip_ins, skip_ft, self.skip_in_ops):
                if skip_flag:
                    if self.up_type != "deconv":
                        h += skip_in_op(F.interpolate(ft, size=(ht, wt), mode=self.up_type))
                    else:
                        scale = wt // ft.size()[-1]
                        h += skip_in_op(getattr(self, f"skip_deconvx{scale}")(ft))

        if self.conv_type == "post":
            final_out = self.post_conv2(h)
        elif self.conv_type == "pre":
            final_out = self.pre_conv2(h)
        else:
            raise NotImplementedError(self.norm_type)

        # shortcut
        if self.short_cut:
            if self.up_type != "deconv":
                final_out += self.c_sc(F.interpolate(x, scale_factor=2, mode=self.up_type))
            else:
                final_out += self.c_sc(self.deconv_sc(x))

        return h_skip_out, final_out




class Generator(nn.Module):
    def __init__(self, gf_dim, bottom_width):
        super(Generator, self).__init__()
        self.ch = gf_dim
        self.bottom_width = bottom_width
        self.l1 = nn.Linear(self.ch, (self.bottom_width ** 2) * self.ch)
        self.cell1 = GCell(self.ch, self.ch, num_skip_in=0)
        self.cell2 = GCell(self.ch, self.ch, num_skip_in=1)
        self.cell3 = GCell(self.ch, self.ch, num_skip_in=2)
        self.to_rgb = nn.Sequential(
            nn.BatchNorm2d(self.ch),
            nn.ReLU(),
            nn.Conv2d(self.ch, 3, 3, 1, 1),
            nn.Tanh(),
        )

    def set_arch(self, arch_id, cur_stage):
        if not isinstance(arch_id, list):
            arch_id = arch_id.to("cpu").numpy().tolist()
        arch_id = [int(x) for x in arch_id]
        self.cur_stage = cur_stage
        arch_stage1 = arch_id[:4]
        self.cell1.set_arch(
            conv_id=arch_stage1[0],
            norm_id=arch_stage1[1],
            up_id=arch_stage1[2],
            short_cut_id=arch_stage1[3],
            skip_ins=[],
        )
        if cur_stage >= 1:
            arch_stage2 = arch_id[4:9]
            self.cell2.set_arch(
                conv_id=arch_stage2[0],
                norm_id=arch_stage2[1],
                up_id=arch_stage2[2],
                short_cut_id=arch_stage2[3],
                skip_ins=arch_stage2[4],
            )

        if cur_stage == 2:
            arch_stage3 = arch_id[9:]
            self.cell3.set_arch(
                conv_id=arch_stage3[0],
                norm_id=arch_stage3[1],
                up_id=arch_stage3[2],
                short_cut_id=arch_stage3[3],
                skip_ins=arch_stage3[4],
            )

    def forward(self, z):
        h = self.l1(z).view(-1, self.ch, self.bottom_width, self.bottom_width)
        h1_skip_out, h1 = self.cell1(h)
        if self.cur_stage == 0:
            return self.to_rgb(h1)
        
        h2_skip_out, h2 = self.cell2(h1, (h1_skip_out,))
        # downsample to make consistent input for to_prob model
        h2_skip_out = _downsample(h2_skip_out)
        h2 = _downsample(h2)
        if self.cur_stage == 1:
            return self.to_rgb(h2)
        
        h1_skip_out = _downsample(h1_skip_out)
        h3_skip_out, h3 = self.cell3(h2, (h1_skip_out, h2_skip_out))
        h3 = _downsample(h3)
        if self.cur_stage == 2:
            return self.to_rgb(h3)
        

# Discriminator Design
- Block design
- Cell design
- Network design

In [3]:
# Discriminator action types -- removed upsample
D_CONV_TYPE = {0: "post", 1: "pre"}
D_NORM_TYPE = {0: None, 1: "bn", 2: "in"}
#UP_TYPE = {0: "bilinear", 1: "nearest", 2: "deconv"}
D_SHORT_CUT_TYPE = {0: False, 1: True}
D_SKIP_TYPE = {0: False, 1: True}


# pre-conv block for Discriminator
class PreDisBlock(nn.Module):
    def __init__(self, in_channels, out_channels, ksize=3):
        super(PreDisBlock, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.inn = nn.InstanceNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, ksize, padding=ksize // 2)

    def set_arch(self, norm_id):
        self.norm_type = NORM_TYPE[norm_id]

    def forward(self, x):
        # norm
        if self.norm_type:
            if self.norm_type == "bn":
                h = self.bn(x)
            elif self.norm_type == "in":
                h = self.inn(x)
            else:
                raise NotImplementedError(self.norm_type)
        else:
            h = x

        # activation
        h = nn.ReLU()(h)

        # conv
        out = self.conv(h)
        return out

# post-conv block for Discriminator
class PostDisBlock(nn.Module):
    def __init__(self, in_channels, out_channels, ksize=3):
        super(PostDisBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, ksize, padding=ksize // 2)
        self.bn = nn.BatchNorm2d(out_channels)
        self.inn = nn.InstanceNorm2d(out_channels)
        
    def set_arch(self, norm_id):
        self.norm_type = NORM_TYPE[norm_id]

    def forward(self, x):
        # whether this is a upsample block
        h = x

        # conv
        h = self.conv(h)

        # norm
        if self.norm_type:
            if self.norm_type == "bn":
                h = self.bn(h)
            elif self.norm_type == "in":
                h = self.inn(h)
            else:
                raise NotImplementedError(self.norm_type)

        # activation
        out = nn.ReLU()(h)

        return out


class DCell(nn.Module):
    def __init__(self, in_channels, out_channels, num_skip_in, ksize=3):
        super(DCell, self).__init__()

        self.post_conv1 = PostDisBlock(in_channels, out_channels, ksize=ksize)
        self.pre_conv1 = PreDisBlock(in_channels, out_channels, ksize=ksize)

        self.post_conv2 = PostDisBlock(out_channels, out_channels, ksize=ksize)
        self.pre_conv2 = PreDisBlock(out_channels, out_channels, ksize=ksize)
        
        self.c_sc = nn.Conv2d(in_channels, out_channels, kernel_size=1)

        # skip_in
        self.skip_deconvx2 = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.skip_deconvx4 = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2),
        )

        self.num_skip_in = num_skip_in
        if num_skip_in:
            self.skip_in_ops = nn.ModuleList([
                    nn.Conv2d(in_channels, out_channels, kernel_size=1)
                    for _ in range(num_skip_in)   
            ])

    # removed up_id
    #def set_arch(self, conv_id, norm_id, up_id, short_cut_id, skip_ins):
    def set_arch(self, conv_id, norm_id, short_cut_id, skip_ins):
        self.post_conv1.set_arch(norm_id)
        self.pre_conv1.set_arch(norm_id)
        self.post_conv2.set_arch(norm_id)
        self.pre_conv2.set_arch(norm_id)

        if self.num_skip_in:
            self.skip_ins = [0 for _ in range(self.num_skip_in)]
            for skip_idx, skip_in in enumerate(decimal2binary(skip_ins)[::-1]):
                self.skip_ins[-(skip_idx + 1)] = int(skip_in)

        self.conv_type = CONV_TYPE[conv_id]
        self.short_cut = SHORT_CUT_TYPE[short_cut_id]

    def forward(self, x, skip_ft=None):
        residual = x

        # first conv
        if self.conv_type == "post":
            h = self.post_conv1(residual)
        elif self.conv_type == "pre":
            h = self.pre_conv1(residual)
        else:
            raise NotImplementedError(self.norm_type)
        _, _, ht, wt = h.size()
        h_skip_out = h
        # second conv
        if self.num_skip_in:
            assert len(self.skip_in_ops) == len(self.skip_ins)
            for skip_flag, ft, skip_in_op in zip(self.skip_ins, skip_ft, self.skip_in_ops):
                if skip_flag:
                    if self.up_type != "deconv":
                        h += skip_in_op(F.interpolate(ft, size=(ht, wt), mode=self.up_type))
                    else:
                        scale = wt // ft.size()[-1]
                        h += skip_in_op(getattr(self, f"skip_deconvx{scale}")(ft))

        if self.conv_type == "post":
            final_out = self.post_conv2(h)
        elif self.conv_type == "pre":
            final_out = self.pre_conv2(h)
        else:
            raise NotImplementedError(self.norm_type)

        # shortcut -- 여기도 upsample 사용 안하도록 변경
        if self.short_cut:
            final_out += self.c_sc(x)
            
        return h_skip_out, final_out


class Discriminator(nn.Module):
    def __init__(self, bottom_width, df_dim):
        super(Discriminator, self).__init__()
        self.ch = df_dim
        self.bottom_width = bottom_width
        self.l1 = nn.Conv2d(3, self.ch, kernel_size=3) #cifar-10: 3x32x32
        self.cell1 = DCell(self.ch, self.ch, num_skip_in=0)
        self.cell2 = DCell(self.ch, self.ch, num_skip_in=1)
        self.cell3 = DCell(self.ch, self.ch, num_skip_in=2)
        # to_rgb -> to_prob
        self.to_prob = nn.Sequential(
            nn.BatchNorm2d(self.ch),
            nn.ReLU(),
            nn.Conv2d(self.ch, 1, 3, 1, 1),
            nn.Flatten(),
            #nn.utils.spectral_norm(nn.Linear(3600, 1, bias=False)),
            nn.utils.spectral_norm(nn.Linear(900, 1, bias=False)),
        )

    def set_arch(self, arch_id, cur_stage):
        if not isinstance(arch_id, list):
            arch_id = arch_id.to("cpu").numpy().tolist()
        arch_id = [int(x) for x in arch_id]
        self.cur_stage = cur_stage
        arch_stage1 = arch_id[:3]
        self.cell1.set_arch(
            conv_id=arch_stage1[0],
            norm_id=arch_stage1[1],
            short_cut_id=arch_stage1[2],
            skip_ins=[],
        )
        if cur_stage >= 1:
            arch_stage2 = arch_id[3:8]
            self.cell2.set_arch(
                conv_id=arch_stage2[0],
                norm_id=arch_stage2[1],
                short_cut_id=arch_stage2[2],
                skip_ins=arch_stage2[3],
            )

        if cur_stage == 2:
            arch_stage3 = arch_id[8:]
            self.cell3.set_arch(
                conv_id=arch_stage3[0],
                norm_id=arch_stage3[1],
                short_cut_id=arch_stage3[2],
                skip_ins=arch_stage3[3],
            )

    def forward(self, x):
        h = self.l1(x)
        
        h1_skip_out, h1 = self.cell1(h)
        if self.cur_stage == 0:
            return self.to_prob(h1)
        
        h2_skip_out, h2 = self.cell2(h1, (h1_skip_out,))
        # downsample to make consistent input for to_prob model
        h2_skip_out = _downsample(h2_skip_out)
        h2 = _downsample(h2)
        if self.cur_stage == 1:
            return self.to_prob(h2)
        
        h1_skip_out = _downsample(h1_skip_out)
        h3_skip_out, h3 = self.cell3(h2, (h1_skip_out, h2_skip_out))
        h3 = _downsample(h3)
        if self.cur_stage == 2:
            return self.to_prob(h3)
        

# Controller Design


In [4]:
class Controller(nn.Module):
    def __init__(self, f_dim, cur_stage):
        """
        init
        :param args:
        :param cur_stage: varies from 0 to ...
        """
        super(Controller, self).__init__()
        self.hid_size = f_dim
        self.cur_stage = cur_stage
        self.lstm = torch.nn.LSTMCell(self.hid_size, self.hid_size)
        if cur_stage:
            self.tokens = [
                len(CONV_TYPE),
                len(NORM_TYPE),
                len(UP_TYPE),
                len(SHORT_CUT_TYPE),
                len(SKIP_TYPE) ** cur_stage,
            ]
        else:
            self.tokens = [
                len(CONV_TYPE),
                len(NORM_TYPE),
                len(UP_TYPE),
                len(SHORT_CUT_TYPE),
            ]
        self.encoder = nn.Embedding(sum(self.tokens), self.hid_size)
        self.decoders = nn.ModuleList(
            [nn.Linear(self.hid_size, token) for token in self.tokens]
        )

    def initHidden(self, batch_size):
        return torch.zeros(batch_size, self.hid_size, requires_grad=False).cuda()

    def forward(self, x, hidden, index):
        if index == 0:
            embed = x
        else:
            embed = self.encoder(x)
        hx, cx = self.lstm(embed, hidden)

        # decode
        logit = self.decoders[index](hx)

        return logit, (hx, cx)

    def sample(self, batch_size, with_hidden=False, prev_hiddens=None, prev_archs=None):
        x = self.initHidden(batch_size)

        if prev_hiddens:
            assert prev_archs
            prev_hxs, prev_cxs = prev_hiddens
            selected_idx = np.random.choice(
                len(prev_archs), batch_size
            )  # TODO: replace=False
            selected_idx = [int(x) for x in selected_idx]

            selected_archs = []
            selected_hxs = []
            selected_cxs = []

            for s_idx in selected_idx:
                selected_archs.append(prev_archs[s_idx].unsqueeze(0))
                selected_hxs.append(prev_hxs[s_idx].unsqueeze(0))
                selected_cxs.append(prev_cxs[s_idx].unsqueeze(0))
            selected_archs = torch.cat(selected_archs, 0)
            hidden = (torch.cat(selected_hxs, 0), torch.cat(selected_cxs, 0))
        else:
            hidden = (self.initHidden(batch_size), self.initHidden(batch_size))
        entropies = []
        actions = []
        selected_log_probs = []
        for decode_idx in range(len(self.decoders)):
            logit, hidden = self.forward(x, hidden, decode_idx)
            prob = F.softmax(logit, dim=-1)  # bs * logit_dim
            log_prob = F.log_softmax(logit, dim=-1)
            entropies.append(-(log_prob * prob).sum(1, keepdim=True))  # bs * 1
            action = prob.multinomial(1)  # batch_size * 1
            actions.append(action)
            selected_log_prob = log_prob.gather(1, action.data)  # batch_size * 1
            selected_log_probs.append(selected_log_prob)

            x = action.view(batch_size) + sum(self.tokens[:decode_idx])
            x = x.requires_grad_(False)

        archs = torch.cat(actions, -1)  # batch_size * len(self.decoders)
        selected_log_probs = torch.cat(
            selected_log_probs, -1
        )  # batch_size * len(self.decoders)
        entropies = torch.cat(entropies, 0)  # bs * 1

        if prev_hiddens:
            archs = torch.cat([selected_archs, archs], -1)

        if with_hidden:
            return archs, selected_log_probs, entropies, hidden

        return archs, selected_log_probs, entropies
    
    
    
class D_Controller(nn.Module):
    def __init__(self, f_dim, cur_stage):
        """
        init
        :param args:
        :param cur_stage: varies from 0 to ...
        """
        super(D_Controller, self).__init__()
        self.hid_size = f_dim
        self.cur_stage = cur_stage
        self.lstm = torch.nn.LSTMCell(self.hid_size, self.hid_size)
        if cur_stage:
            self.tokens = [
                len(CONV_TYPE),
                len(NORM_TYPE),
                len(SHORT_CUT_TYPE),
                len(SKIP_TYPE) ** cur_stage,
            ]
        else:
            self.tokens = [
                len(CONV_TYPE),
                len(NORM_TYPE),
                len(SHORT_CUT_TYPE),
            ]
        self.encoder = nn.Embedding(sum(self.tokens), self.hid_size)
        self.decoders = nn.ModuleList(
            [nn.Linear(self.hid_size, token) for token in self.tokens]
        )

    def initHidden(self, batch_size):
        return torch.zeros(batch_size, self.hid_size, requires_grad=False).cuda()

    def forward(self, x, hidden, index):
        if index == 0:
            embed = x
        else:
            embed = self.encoder(x)
        hx, cx = self.lstm(embed, hidden)

        # decode
        logit = self.decoders[index](hx)

        return logit, (hx, cx)

    def sample(self, batch_size, with_hidden=False, prev_hiddens=None, prev_archs=None):
        x = self.initHidden(batch_size)

        if prev_hiddens:
            assert prev_archs
            prev_hxs, prev_cxs = prev_hiddens
            selected_idx = np.random.choice(
                len(prev_archs), batch_size
            )  # TODO: replace=False
            selected_idx = [int(x) for x in selected_idx]

            selected_archs = []
            selected_hxs = []
            selected_cxs = []

            for s_idx in selected_idx:
                selected_archs.append(prev_archs[s_idx].unsqueeze(0))
                selected_hxs.append(prev_hxs[s_idx].unsqueeze(0))
                selected_cxs.append(prev_cxs[s_idx].unsqueeze(0))
            selected_archs = torch.cat(selected_archs, 0)
            hidden = (torch.cat(selected_hxs, 0), torch.cat(selected_cxs, 0))
        else:
            hidden = (self.initHidden(batch_size), self.initHidden(batch_size))
        entropies = []
        actions = []
        selected_log_probs = []
        for decode_idx in range(len(self.decoders)):
            logit, hidden = self.forward(x, hidden, decode_idx)
            prob = F.softmax(logit, dim=-1)  # bs * logit_dim
            log_prob = F.log_softmax(logit, dim=-1)
            entropies.append(-(log_prob * prob).sum(1, keepdim=True))  # bs * 1
            action = prob.multinomial(1)  # batch_size * 1
            actions.append(action)
            selected_log_prob = log_prob.gather(1, action.data)  # batch_size * 1
            selected_log_probs.append(selected_log_prob)

            x = action.view(batch_size) + sum(self.tokens[:decode_idx])
            x = x.requires_grad_(False)

        archs = torch.cat(actions, -1)  # batch_size * len(self.decoders)
        selected_log_probs = torch.cat(
            selected_log_probs, -1
        )  # batch_size * len(self.decoders)
        entropies = torch.cat(entropies, 0)  # bs * 1

        if prev_hiddens:
            archs = torch.cat([selected_archs, archs], -1)

        if with_hidden:
            return archs, selected_log_probs, entropies, hidden

        return archs, selected_log_probs, entropies

# init
- define G_controller & D_controller
- define initial G & D

In [5]:
# some cute utility functions
def decimal2binary(n):
    return bin(n).replace("0b", "")

def _downsample(x):
    # Downsample (Mean Avg Pooling with 2x2 kernel)
    return nn.AvgPool2d(kernel_size=2)(x)

def weights_init(m, init_type='xavier'):
    classname = m.__class__.__name__
    if classname.find("Conv2d") != -1:
        if init_type == "normal":
            nn.init.normal_(m.weight.data, 0.0, 0.02)
        elif init_type == "orth":
            nn.init.orthogonal_(m.weight.data)
        elif init_type == "xavier":
            nn.init.xavier_uniform_(m.weight.data, 1.0)
        else:
            raise NotImplementedError("{}: unknown inital type".format(init_type))
    elif classname.find("BatchNorm2d") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

# simple debug

In [6]:
# define nets, contents: def create_shared_gan(args, weights_init)
# arguments from cfg.py
f_dim = 256
bottom_width = 16 # what is this? --> determine minimal output size
beta1 = 0.0
beta2 = 0.9
g_lr = 0.0002
d_lr = 0.0002
g_spectral_norm = False
d_spectral_norm = True # applied manually
########################
gen_net = Generator(f_dim, bottom_width)
dis_net = Discriminator(f_dim, bottom_width)
gen_net.apply(weights_init)
dis_net.apply(weights_init)
gen_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, gen_net.parameters()), g_lr, (beta1, beta2))
dis_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, dis_net.parameters()), d_lr, (beta1, beta2))

# set controller && its optimizer
# def create_ctrler(args, cur_stage, weights_init)
cur_stage = 0
cont_f_dim = 128
ctrl_lr = 0.00035
########################
G_controller = Controller(cont_f_dim, cur_stage).cuda()
G_controller.apply(weights_init)
G_controller_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, G_controller.parameters()), ctrl_lr, (beta1, beta2))
G_prev_archs = None
G_prev_hiddens = None

D_controller = D_Controller(cont_f_dim, cur_stage).cuda()
D_controller.apply(weights_init)
D_controller_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, D_controller.parameters()), ctrl_lr, (beta1, beta2))
D_prev_archs = None
D_prev_hiddens = None

In [7]:
# test
# sample G arch
G_arch, _, _ = G_controller.sample(1)
G_arch = G_arch[0]
# set G arch
gen_net.set_arch(G_arch, 0)
# input
z = torch.FloatTensor(np.random.normal(0, 1, (1, f_dim)))
# output
o = gen_net(z)
print(o.shape) # cifar-10 image shape

# sample D arch
D_arch, _, _ = D_controller.sample(1)
D_arch = D_arch[0]
print(D_arch)
# set D arch
dis_net.set_arch(D_arch, 0)
# input: G output
dis_net(o)

torch.Size([1, 3, 32, 32])
tensor([0, 2, 1], device='cuda:0')


tensor([[0.4549]], grad_fn=<MmBackward>)

In [8]:
from itertools import product
archs = product([0, 1], [0, 1, 2], [0, 1])
for arch in list(archs):
    dis_net.set_arch(list(arch), 0)
    try:
        dis_net(o)
    except:
        1#print('arch {} has shape error'.format(arch))
    else:
        print('arch {} has no error'.format(arch))
#for i in range

arch (0, 0, 0) has no error
arch (0, 0, 1) has no error
arch (0, 1, 0) has no error
arch (0, 1, 1) has no error
arch (0, 2, 0) has no error
arch (0, 2, 1) has no error
arch (1, 0, 0) has no error
arch (1, 0, 1) has no error
arch (1, 1, 0) has no error
arch (1, 1, 1) has no error
arch (1, 2, 0) has no error
arch (1, 2, 1) has no error


# Training Functions from functions.py
- train_shared: train shared GAN (supernet)
- train_controller
- get_topK_archs

In [9]:
def train_shared(
    # commons
    params,
    train_loader,
    # GAN
    gen_net: nn.Module,
    dis_net: nn.Module,
    gen_optimizer,
    dis_optimizer,
    g_loss_history,
    d_loss_history,
    # controllers
    G_controller,
    D_controller,
    G_prev_hiddens=None,
    G_prev_archs=None,
    D_prev_hiddens=None,
    D_prev_archs=None,
):
    dynamic_reset = False
    logger.info("=> train shared GAN...")
    step = 0
    gen_step = 0

    # train mode
    gen_net.train()
    dis_net.train()

    # eval mode
    G_controller.eval()
    D_controller.eval()
    for epoch in range(params['shared_epoch']):
        for iter_idx, (imgs, _) in enumerate(train_loader):

            # sample an G arch
            G_arch = G_controller.sample(1, prev_hiddens=G_prev_hiddens, prev_archs=G_prev_archs)[0][0]
            gen_net.set_arch(G_arch, G_controller.cur_stage)
            
            # sample and D arch
            D_arch = D_controller.sample(1, prev_hiddens=D_prev_hiddens, prev_archs=D_prev_archs)[0][0]
            dis_net.set_arch(D_arch, D_controller.cur_stage)
            # dis_net.cur_stage = G_controller.cur_stage # original code
            
            # Adversarial ground truths
            real_imgs = imgs.type(torch.cuda.FloatTensor)

            # Sample noise as generator input
            z = torch.cuda.FloatTensor(np.random.normal(0, 1, (imgs.shape[0], params['latent_dim'])))
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            dis_optimizer.zero_grad()

            real_validity = dis_net(real_imgs)
            fake_imgs = gen_net(z).detach()
            assert fake_imgs.size() == real_imgs.size(), print(
                f"fake image size is {fake_imgs.size()}, "
                f"while real image size is {real_imgs.size()}"
            )
            fake_validity = dis_net(fake_imgs)

            # cal loss
            d_loss = torch.mean(nn.ReLU(inplace=True)(1.0 - real_validity)) + torch.mean(nn.ReLU(inplace=True)(1 + fake_validity))
            d_loss.backward()
            dis_optimizer.step()

            # add to window
            d_loss_history.push(d_loss.item())

            # -----------------
            #  Train Generator
            # -----------------
            if step % params['n_critic'] == 0:
                gen_optimizer.zero_grad()
                gen_z = torch.cuda.FloatTensor(np.random.normal(0, 1, (params['G_batch_size'], params['latent_dim'])))
                gen_imgs = gen_net(gen_z)
                fake_validity = dis_net(gen_imgs)

                # cal loss
                g_loss = -torch.mean(fake_validity)
                g_loss.backward()
                gen_optimizer.step()

                # add to window
                g_loss_history.push(g_loss.item())
                gen_step += 1

            # verbose
            if gen_step and iter_idx % params['print_freq'] == 0:
                logger.info(
                    "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                    % (
                        epoch,
                        params['shared_epoch'],
                        iter_idx % len(train_loader),
                        len(train_loader),
                        d_loss.item(),
                        g_loss.item(),
                    )
                )

            # check window
            if g_loss_history.is_full():
                if (g_loss_history.get_var() < params['dynamic_reset_threshold']
                    or d_loss_history.get_var() < params['dynamic_reset_threshold']):
                    dynamic_reset = True
                    logger.info("=> dynamic resetting triggered")
                    g_loss_history.clear()
                    d_loss_history.clear()
                    return dynamic_reset

            step += 1

    return dynamic_reset

    
def train_G_controller_(
    G_params,
    G_controller,
    G_controler_opt,
    gen_net,
    G_prev_hiddens,
    G_prev_archs,
    search_iter,
    writer
):
    logger.info("=> train G_controller...")
    baseline = None

    # G_controller train mode
    G_controller.train()

    # eval mode
    gen_net.eval()

    mean_rewards = []
    mean_advs = []
    mean_entropies = []
    cur_stage = G_controller.cur_stage
    for step in range(G_params['ctrl_step']):
        G_archs, selected_log_probs, entropies = G_controller.sample(G_params['ctrl_sample_batch'],
                                                                       prev_hiddens=G_prev_hiddens,
                                                                       prev_archs=G_prev_archs)
        cur_batch_rewards = []
        for arch in G_archs:
            logger.info(f"arch: {arch}")
            gen_net.set_arch(arch, cur_stage)
            #############################
            # computing Inception Score #
            #############################
            is_score = get_is(gen_net, G_params['rl_num_eval_img'])
            logger.info(f"get Inception score of {is_score}")
            cur_batch_rewards.append(is_score)
        cur_batch_rewards = torch.tensor(cur_batch_rewards, requires_grad=False).cuda()
        cur_batch_rewards = (cur_batch_rewards.unsqueeze(-1) + G_params['entropy_coeff'] * entropies)  # bs * 1
        if baseline is None:
            baseline = cur_batch_rewards
        else:
            baseline = (G_params['baseline_decay'] * baseline.detach() + (1 - G_params['baseline_decay']) * cur_batch_rewards)
        adv = cur_batch_rewards - baseline

        # policy loss
        G_loss = -selected_log_probs * adv
        G_loss = G_loss.sum()

        # update controller
        G_controller_opt.zero_grad()
        G_loss.backward()
        G_controller_opt.step()

        # write
        mean_rewards.append(cur_batch_rewards.mean().item())
        mean_advs.append(adv.mean().item())
        mean_entropies.append(entropies.mean().item())
    writer.add_scalar("G_controller/loss", G_loss.item(), search_iter)
    writer.add_scalar("G_controller/reward", np.mean(mean_rewards), search_iter)
    writer.add_scalar("G_controller/entropy", np.mean(mean_entropies), search_iter)
    writer.add_scalar("G_controller/adv", np.mean(mean_advs), search_iter)


def train_D_controller_(
    D_params,
    D_controller,
    D_controler_opt,
    dis_net,
    D_prev_hiddens,
    D_prev_archs,
    search_iter,
    writer,
    gen_net
):
    logger.info("=> train D_controller...")
    baseline = None
    
    ## ready
    D_controller.train()
    # eval mode
    dis_net.eval()
    gen_net.eval()
    
    mean_rewards = []
    mean_advs = []
    mean_entropies = []
    cur_stage = D_controller.cur_stage
    for step in range(D_params['ctrl_step']):
        D_archs, selected_log_probs, entropies = D_controller.sample(D_params['ctrl_sample_batch'],
                                                                     prev_hiddens=D_prev_hiddens,
                                                                     prev_archs=D_prev_archs)
        cur_batch_rewards = []
        for arch in D_archs:
            logger.info(f"arch: {arch}")
            dis_net.set_arch(arch, cur_stage)
            ##################################
            # discrimination power: Accuracy #
            ##################################
            # from train_shared_GAN
            # CUDA Memory problem -- solved
            cur_arch_rewards = []
            for iter_idx, (imgs, _) in enumerate(train_loader):
                real_imgs = imgs.type(torch.cuda.FloatTensor)
                real_validity = dis_net(real_imgs)
                # Sample noise as generator input
                
                z = torch.cuda.FloatTensor(np.random.normal(0, 1, (imgs.shape[0], D_params['latent_dim'])))
                fake_imgs = gen_net(z).detach()
                fake_validity = dis_net(fake_imgs)
                cbr = torch.mean(nn.ReLU(inplace=True)(1.0 - real_validity)) + torch.mean(nn.ReLU(inplace=True)(1 + fake_validity))
                cur_arch_rewards.append(cbr.item())
            cur_batch_rewards.append(np.mean(cur_arch_rewards))
        cur_batch_rewards = torch.tensor(cur_batch_rewards, requires_grad=False).cuda()
        cur_batch_rewards = (cur_batch_rewards.unsqueeze(-1) + D_params['entropy_coeff'] * entropies)  # bs * 1
        if baseline is None:
            baseline = cur_batch_rewards
        else:
            baseline = (D_params['baseline_decay'] * baseline.detach() + (1 - D_params['baseline_decay']) * cur_batch_rewards)
        adv = cur_batch_rewards - baseline

        # policy loss
        D_loss = -selected_log_probs * adv
        D_loss = D_loss.sum()

        # update controller
        D_controller_opt.zero_grad()
        D_loss.backward()
        D_controller_opt.step()

        # write
        mean_rewards.append(cur_batch_rewards.mean().item())
        mean_advs.append(adv.mean().item())
        mean_entropies.append(entropies.mean().item())
        
    writer.add_scalar("D_controller/loss", D_loss.item(), search_iter)
    writer.add_scalar("D_controller/reward", np.mean(mean_rewards), search_iter)
    writer.add_scalar("D_controller/entropy", np.mean(mean_entropies), search_iter)
    writer.add_scalar("D_controller/adv", np.mean(mean_advs), search_iter)
    
    
def get_is(gen_net: nn.Module, num_img):
    """
    Get inception score.
    :param args:
    :param gen_net:
    :param num_img:
    :return: Inception score
    """
    eval_batch_size = 100
    latent_dim = f_dim
    
    # eval mode
    gen_net = gen_net.eval()

    eval_iter = num_img // eval_batch_size
    img_list = list()
    for _ in range(eval_iter):
        z = torch.cuda.FloatTensor(np.random.normal(0, 1, (eval_batch_size, latent_dim)))

        # Generate a batch of images
        gen_imgs = (
            gen_net(z)
            .mul_(127.5)
            .add_(127.5)
            .clamp_(0.0, 255.0)
            .permute(0, 2, 3, 1)
            .to("cpu", torch.uint8)
            .numpy()
        )
        img_list.extend(list(gen_imgs))

    # get inception score
    logger.info("calculate Inception score...")
    x = np.asarray(img_list)
    trans = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    xs = []
    for d in x:
        d = Image.fromarray(d)
        d = trans(d)
        xs.append(torch.unsqueeze(d, 0))

    b = torch.Tensor(10000, 3, 32, 32)
    torch.cat(xs, out=b)
    mean, std = ist.get_IS(b)
    return mean


def get_topk_G_arch_hidden(params, G_controller, gen_net, G_prev_archs, G_prev_hiddens):
    """
    ~
    :param args:
    :param controller:
    :param gen_net:
    :param prev_archs: previous architecture
    :param prev_hiddens: previous hidden vector
    :return: a list of topk archs and hiddens.
    """
    topk = params['topk']
    num_candidate = params['num_candidate']
    logger.info(
        f"=> G: get top{topk} archs out of {num_candidate} candidate archs..."
    )
    assert num_candidate >= topk
    G_controller.eval()
    cur_stage = G_controller.cur_stage
    G_archs, _, _, hiddens = G_controller.sample(num_candidate, with_hidden=True, prev_archs=G_prev_archs, prev_hiddens=G_prev_hiddens)
    hxs, cxs = hiddens
    arch_idx_perf_table = {}
    for arch_idx in range(len(G_archs)):
        logger.info(f"arch: {G_archs[arch_idx]}")
        gen_net.set_arch(G_archs[arch_idx], cur_stage)
        is_score = get_is(gen_net, params['rl_num_eval_img'])
        logger.info(f"get Inception score of {is_score}")
        arch_idx_perf_table[arch_idx] = is_score
    topk_arch_idx_perf = sorted(arch_idx_perf_table.items(), key=operator.itemgetter(1))[::-1][:topk]
    topk_archs = []
    topk_hxs = []
    topk_cxs = []
    logger.info(f"top{topk} archs:")
    for arch_idx_perf in topk_arch_idx_perf:
        logger.info(arch_idx_perf)
        arch_idx = arch_idx_perf[0]
        topk_archs.append(G_archs[arch_idx])
        topk_hxs.append(hxs[arch_idx].detach().requires_grad_(False))
        topk_cxs.append(cxs[arch_idx].detach().requires_grad_(False))

    return topk_archs, (topk_hxs, topk_cxs)


def get_topk_D_arch_hidden(params, D_controller, dis_net, D_prev_archs, D_prev_hiddens,
                                   G_controller, gen_net, G_prev_archs, G_prev_hiddens):
    """
    ~
    :param args:
    :param controller:
    :param gen_net:
    :param prev_archs: previous architecture
    :param prev_hiddens: previous hidden vector
    :return: a list of topk archs and hiddens.
    """
    topk = params['topk']
    num_candidate = params['num_candidate']
    logger.info(f"=> D: get top{topk} archs out of {num_candidate} candidate archs...")
    
    assert num_candidate >= topk
    gen_net.eval()
    D_controller.eval()
    cur_stage = D_controller.cur_stage
    D_archs, _, _, hiddens = D_controller.sample(num_candidate, with_hidden=True, prev_archs=D_prev_archs, prev_hiddens=D_prev_hiddens)
    hxs, cxs = hiddens
    arch_idx_perf_table = {}
    # to get topk discriminators, test performance for all topk generators
    for arch_idx in range(len(D_archs)):
        logger.info(f"arch: {D_archs[arch_idx]}")
        dis_net.set_arch(D_archs[arch_idx], cur_stage)
        
        # test for all top k generator architectures
        cur_arch_rewards = []
        for G_arch_idx in range(len(G_prev_archs)):
            gen_net.set_arch(G_prev_archs[G_arch_idx], G_controller.cur_stage)
            cur_G_arch_rewards = []
            for iter_idx, (imgs, _) in enumerate(train_loader):
                real_imgs = imgs.type(torch.cuda.FloatTensor)
                real_validity = dis_net(real_imgs)
                
                # Sample noise as generator input
                z = torch.cuda.FloatTensor(np.random.normal(0, 1, (imgs.shape[0], params['latent_dim'])))
                fake_imgs = gen_net(z).detach()
                fake_validity = dis_net(fake_imgs)
                cbr = torch.mean(nn.ReLU(inplace=True)(1.0 - real_validity)) + torch.mean(nn.ReLU(inplace=True)(1 + fake_validity))
                cur_G_arch_rewards.append(cbr.item())
                # shorter validation
                if iter_idx > 100:
                    break
            cur_arch_rewards.append(np.mean(cur_G_arch_rewards))
            
        arch_idx_perf_table[arch_idx] = np.mean(cur_arch_rewards)
    topk_arch_idx_perf = sorted(arch_idx_perf_table.items(), key=operator.itemgetter(1))[::-1][:topk]
    topk_archs = []
    topk_hxs = []
    topk_cxs = []
    logger.info(f"top{topk} archs:")
    for arch_idx_perf in topk_arch_idx_perf:
        logger.info(arch_idx_perf)
        arch_idx = arch_idx_perf[0]
        topk_archs.append(D_archs[arch_idx])
        topk_hxs.append(hxs[arch_idx].detach().requires_grad_(False))
        topk_cxs.append(cxs[arch_idx].detach().requires_grad_(False))

    return topk_archs, (topk_hxs, topk_cxs)

- dataloader

In [10]:
class ImageDataset(object):
    def __init__(self, name, dpath, cur_img_size=None):
        # parameters from cfg.py
        data_path = './data'
        dis_batch_size = 64
        num_workers = 8
        img_size = cur_img_size if cur_img_size else 32 # cifar-10
        if name.lower() == "cifar10":
            Dt = datasets.CIFAR10
            transform = transforms.Compose([
                transforms.Resize(img_size),
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
            n_classes = 10
        elif name.lower() == "stl10":
            Dt = datasets.STL10
            transform = transforms.Compose([
                transforms.Resize(img_size),
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        else:
            raise NotImplementedError("Unknown dataset: {}".format(name))

        if name.lower() == "stl10":
            self.train = torch.utils.data.DataLoader(
                Dt(
                    root=data_path,
                    split="train+unlabeled",
                    transform=transform,
                    download=True,
                ),
                batch_size=dis_batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=True,
            )

            self.valid = torch.utils.data.DataLoader(
                Dt(root=data_path, split="test", transform=transform),
                batch_size=dis_batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=True,
            )

            self.test = self.valid
        else:
            self.train = torch.utils.data.DataLoader(
                Dt(root=data_path, train=True, transform=transform, download=True),
                batch_size=dis_batch_size,
                shuffle=True,
                num_workers=num_workers,
                pin_memory=True,
            )

            self.valid = torch.utils.data.DataLoader(
                Dt(root=data_path, train=False, transform=transform),
                batch_size=dis_batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=True,
            )

            self.test = self.valid

- custom validation function

In [11]:
def get_FID(log_paths, gen_net: nn.Module):
    print('calculating FID score...')
    
    fid_stat = './data/fid_stats_cifar10_train.npz'
    num_eval_imgs = 50000
    eval_batch_size = 100
    clean_dir = True

    # eval mode
    gen_net = gen_net.eval()

    # get fid
    fid_buffer_dir = os.path.join(log_paths["sample_path"], "fid_buffer")
    os.makedirs(fid_buffer_dir, exist_ok=True)

    eval_iter = num_eval_imgs // eval_batch_size
    img_list = list()
    for iter_idx in tqdm(range(eval_iter), desc="sample images"):
        z = torch.cuda.FloatTensor(np.random.normal(0, 1, (eval_batch_size, f_dim)))

        # Generate a batch of images
        gen_imgs = (
            gen_net(z)
            .mul_(127.5)
            .add_(127.5)
            .clamp_(0.0, 255.0)
            .permute(0, 2, 3, 1)
            .to("cpu", torch.uint8)
            .numpy()
        )
        for img_idx, img in enumerate(gen_imgs):
            file_name = os.path.join(fid_buffer_dir, f"iter{iter_idx}_b{img_idx}.png")
            imsave(file_name, img)
        img_list.extend(list(gen_imgs))

    # get fid score
    fid_score = fst.get_fid(fid_buffer_dir, fid_stat)

    if clean_dir:
        os.system("rm -r {}".format(fid_buffer_dir))

    return fid_score

# Prepare Experiment
- hyperparameter setting
- create Nets & Controllers

In [12]:
# experiment hyperparameters from cfg.py
exp_name = 'AutoGAN_all'
f_dim = 128
G_batch_size = 128
D_batch_size = 64
max_iter = 50000
dynamic_reset_window = 500
n_critic = 5
val_freq = 20
# set grow step of controller
grow_step1 = 25
grow_step2 = 50
cur_stage = 0

# create shared GAN training parameters
shared_params = {}
shared_params['shared_epoch'] = 15
shared_params['latent_dim'] = f_dim
shared_params['n_critic'] = n_critic
shared_params['G_batch_size'] = G_batch_size
shared_params['print_freq'] = 100
shared_params['dynamic_reset_threshold'] = 1e-3

topk_params = {}
topk_params['topk'] = 5
topk_params['latent_dim'] = f_dim
topk_params['num_candidate'] = 10
topk_params['rl_num_eval_img'] = 5000

# create Controller training parameters
G_cont_params = {}
G_cont_params['ctrl_step'] = 30
G_cont_params['ctrl_sample_batch'] = 1
G_cont_params['entropy_coeff'] = 1e-3 # encourage exploration
G_cont_params['baseline_decay'] = 0.9
G_cont_params['rl_num_eval_img'] = 5000
G_cont_params['latent_dim'] = f_dim

D_cont_params = {}
D_cont_params['ctrl_step'] = 30
D_cont_params['ctrl_sample_batch'] = 1
D_cont_params['entropy_coeff'] = 1e-3
D_cont_params['baseline_decay'] = 0.9
D_cont_params['rl_num_eval_img'] = 5000
D_cont_params['latent_dim'] = f_dim

# create log dir
log_paths = set_log_dir("logs", exp_name)
logger = create_logger(log_paths['log_path'])

# set up tensorboard writer
writer = SummaryWriter('./runs/')

# set up data_loader
dataset = ImageDataset('cifar10', 2 ** (cur_stage + 3))
train_loader = dataset.train

Files already downloaded and verified


# define nets

In [13]:
# define nets, contents: def create_shared_gan(args, weights_init)
# arguments from cfg.py
bottom_width = 16 # what is this? --> determine minimal output size
beta1 = 0.0
beta2 = 0.9
g_lr = 0.0002
d_lr = 0.0002
g_spectral_norm = False
d_spectral_norm = True # applied manually
########################
gen_net = Generator(f_dim, bottom_width).cuda()
dis_net = Discriminator(f_dim, bottom_width).cuda()
gen_net.apply(weights_init)
dis_net.apply(weights_init)
gen_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, gen_net.parameters()), g_lr, (beta1, beta2))
dis_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, dis_net.parameters()), d_lr, (beta1, beta2))
g_loss_history = RunningStats(dynamic_reset_window)
d_loss_history = RunningStats(dynamic_reset_window)

# set controller && its optimizer
# def create_ctrler(args, cur_stage, weights_init)
cur_stage = 0
cont_f_dim = 128
ctrl_lr = 0.00035
########################
G_controller = Controller(cont_f_dim, cur_stage).cuda()
G_controller.apply(weights_init)
G_controller_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, G_controller.parameters()), ctrl_lr, (beta1, beta2))
G_prev_archs = None
G_prev_hiddens = None

D_controller = D_Controller(cont_f_dim, cur_stage).cuda()
D_controller.apply(weights_init)
D_controller_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, D_controller.parameters()), ctrl_lr, (beta1, beta2))
D_prev_archs = None
D_prev_hiddens = None

In [14]:
# train loop
Iscore, Fscore = 0, 300
search_iter = 1
grow_step1 = 15
while Iscore < 8 or Fscore > 15:
    #for search_iter in tqdm(range(int(start_search_iter), int(args.max_search_iter)), desc="search progress"):
    
    logger.info(f"<start search iteration {search_iter}>")
    if search_iter % grow_step1 == 0:
        # save
        if search_iter < grow_step1:
            cur_stage = 0
        elif search_iter < 2 * grow_step1:
            cur_stage = 0
        else:
            cur_stage = 0
            
        #cur_stage = search_iter // grow_step1
        logger.info(f"=> grow to stage {cur_stage}")
        G_prev_archs, G_prev_hiddens = get_topk_G_arch_hidden(topk_params, G_controller, gen_net, G_prev_archs, G_prev_hiddens)
        D_prev_archs, D_prev_hiddens = get_topk_D_arch_hidden(topk_params,
                                                              D_controller, dis_net, D_prev_archs, D_prev_hiddens,
                                                              G_controller, gen_net, G_prev_archs, G_prev_hiddens)
        
        # grow section
        del G_controller, G_controller_opt, D_controller, D_controller_opt
        # originally implemented in create_ctrler(args, cur_stage, weights_init)
        G_controller = Controller(cont_f_dim, cur_stage).cuda()
        G_controller.apply(weights_init)
        G_controller_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, G_controller.parameters()), ctrl_lr, (beta1, beta2))
        
        D_controller = D_Controller(cont_f_dim, cur_stage).cuda()
        D_controller.apply(weights_init)
        D_controller_opt = torch.optim.Adam(filter(lambda p: p.requires_grad, D_controller.parameters()), ctrl_lr, (beta1, beta2))

        dataset = ImageDataset('cifar10', 2 ** (cur_stage + 3))
        train_loader = dataset.train

    # train shared GAN
    dynamic_reset = train_shared(
        # commons
        shared_params,
        train_loader,
        gen_net,
        dis_net,
        gen_optimizer,
        dis_optimizer,
        g_loss_history,
        d_loss_history,
        G_controller,
        D_controller,
        G_prev_hiddens=G_prev_hiddens,
        G_prev_archs=G_prev_archs,
        D_prev_hiddens=D_prev_hiddens,
        D_prev_archs=D_prev_archs,
    )

    # train controllers
    train_G_controller_(
        G_cont_params,
        G_controller,
        G_controller_opt,
        gen_net,
        G_prev_hiddens,
        G_prev_archs,
        search_iter,
        writer
    )

    train_D_controller_(
        D_cont_params,
        D_controller,
        D_controller_opt,
        dis_net,
        D_prev_hiddens,
        D_prev_archs,
        search_iter,
        writer,
        gen_net
    )
    
    mean = get_is(gen_net, 50000)
    fid = get_FID(log_paths, gen_net)
    print(mean, fid)
    
    writer.add_scalar("Inception_score", mean, search_iter)
    writer.add_scalar("Frechet_inception_score", fid, search_iter)
    
    dynamic_reset = True
    if dynamic_reset:
        logger.info("re-initialize share GAN")
        del gen_net, dis_net, gen_optimizer, dis_optimizer
        # originally implemented as create_shared_gan(args, weights_init)
        gen_net = Generator(f_dim, bottom_width).cuda()
        dis_net = Discriminator(f_dim, bottom_width).cuda()
        gen_net.apply(weights_init)
        dis_net.apply(weights_init)
        gen_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, gen_net.parameters()), g_lr, (beta1, beta2))
        dis_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, dis_net.parameters()), d_lr, (beta1, beta2))
        

    save_checkpoint(
        {
            "cur_stage": cur_stage,
            "search_iter": search_iter + 1,
            "gen_model": "shared_gan",
            "dis_model": "shared_gan",
            "controller": "controller",
            "gen_state_dict": gen_net.state_dict(),
            "dis_state_dict": dis_net.state_dict(),
            "G_ctrl_state_dict": G_controller.state_dict(),
            "D_ctrl_state_dict": D_controller.state_dict(),
            "gen_optimizer": gen_optimizer.state_dict(),
            "dis_optimizer": dis_optimizer.state_dict(),
            "G_ctrl_optimizer": G_controller_opt.state_dict(),
            "D_ctrl_optimizer": D_controller_opt.state_dict(),
            "G_prev_archs": G_prev_archs,
            "G_prev_hiddens": G_prev_hiddens,
            "D_prev_archs": D_prev_archs,
            "D_prev_hiddens": D_prev_hiddens,
            "path_helper": log_paths,
        },
        False,
        log_paths["ckpt_path"],
    )
    
    search_iter += 1

writer.close()

<start search iteration 1>
=> train shared GAN...
[Epoch 0/15] [Batch 0/782] [D loss: 2.001403] [G loss: -0.042158]
/home/taegun/research/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
[Epoch 0/15] [Batch 100/782] [D loss: 1.448730] [G loss: 1.000453]
[Epoch 0/15] [Batch 200/782] [D loss: 0.520591] [G loss: 1.487401]
[Epoch 0/15] [Batch 300/782] [D loss: 0.211440] [G loss: 2.061092]
[Epoch 0/15] [Batch 400/782] [D loss: 0.271524] [G loss: 1.531891]
[Epoch 0/15] [Batch 500/782] [D loss: 0.121706] [G loss: 3.445463]
[Epoch 0/15] [Batch 600/782] [D loss: 0.596115] [G loss: 2.571231]
[Epoch 0/15] [Batch 700/782] [D loss: 0.273369] [G loss: 1.862184]
[Epoch 1/15] [Batch 0/782] [D loss: 0.590299] [G loss: 2.747581]
[Epoch 1/15] [Batch 100/782] [D

[Epoch 14/15] [Batch 500/782] [D loss: 0.000000] [G loss: 6.497431]
[Epoch 14/15] [Batch 600/782] [D loss: 0.037844] [G loss: 2.478795]
[Epoch 14/15] [Batch 700/782] [D loss: 0.033639] [G loss: 1.756616]
=> train G_controller...
arch: tensor([1, 1, 0, 1], device='cuda:0')
calculate Inception score...
get Inception score of 1.3271837112357578
arch: tensor([0, 0, 0, 0], device='cuda:0')
calculate Inception score...
get Inception score of 2.512525481536026
arch: tensor([1, 0, 2, 0], device='cuda:0')
calculate Inception score...
get Inception score of 1.0871829828646922
arch: tensor([1, 1, 1, 0], device='cuda:0')
calculate Inception score...
get Inception score of 1.4010251332991666
arch: tensor([1, 2, 2, 1], device='cuda:0')
calculate Inception score...
get Inception score of 1.3883521153451248
arch: tensor([1, 1, 2, 0], device='cuda:0')
calculate Inception score...
get Inception score of 1.2480723524204629
arch: tensor([0, 2, 0, 1], device='cuda:0')
calculate Inception score...
get Incep

calculating FID score...


sample images: 100%|██████████████████████████████████████████████████████████████████| 500/500 [00:15<00:00, 32.58it/s]
re-initialize share GAN
<start search iteration 2>
=> train shared GAN...


1.085603323618227 378.42481556515315


...

2.273084673907533 212.9338351276079


[Epoch 0/15] [Batch 0/782] [D loss: 2.082884] [G loss: 0.025471]
[Epoch 0/15] [Batch 100/782] [D loss: 0.358681] [G loss: 1.335270]
[Epoch 0/15] [Batch 200/782] [D loss: 0.098329] [G loss: 1.825010]
[Epoch 0/15] [Batch 300/782] [D loss: 0.036700] [G loss: 2.195714]
[Epoch 0/15] [Batch 400/782] [D loss: 0.013350] [G loss: 2.074220]
[Epoch 0/15] [Batch 500/782] [D loss: 0.015078] [G loss: 2.260363]
[Epoch 0/15] [Batch 600/782] [D loss: 0.000000] [G loss: 2.600111]
[Epoch 0/15] [Batch 700/782] [D loss: 0.000000] [G loss: 2.077619]
[Epoch 1/15] [Batch 0/782] [D loss: 0.000000] [G loss: 2.135857]
[Epoch 1/15] [Batch 100/782] [D loss: 0.000000] [G loss: 1.922177]
[Epoch 1/15] [Batch 200/782] [D loss: 0.000000] [G loss: 1.971030]
[Epoch 1/15] [Batch 300/782] [D loss: 0.000000] [G loss: 1.864427]
[Epoch 1/15] [Batch 400/782] [D loss: 0.000000] [G loss: 2.002854]
[Epoch 1/15] [Batch 500/782] [D loss: 0.000000] [G loss: 2.046209]
[Epoch 1/15] [Batch 600/782] [D loss: 0.000000] [G loss: 1.910659]

calculate Inception score...
get Inception score of 1.0248675274532875
arch: tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 1,
        0, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 0,
        1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 0, 1, 1, 1, 2, 1], device='cuda:0')
calculate Inception score...
get Inception score of 1.0250850969137
arch: tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 1,
        0, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 0,
        1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 0, 1, 0, 2, 1, 0], device='cuda:0')
calculate Inception score...
get Inception score of 1.0246691780058304
arch: tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 1,
        0, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 0,
        1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 2, 0, 1], device='cuda:0')
calculate Inception score...
get Inception score of 1.0242016000872392

arch: tensor([1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 0, 0, 2, 0,
        0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 1, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1],
       device='cuda:0')
arch: tensor([1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 0, 0, 2, 0,
        0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 1, 1, 1, 2, 0, 1, 2, 1, 1, 2, 1],
       device='cuda:0')
calculate Inception score...


KeyboardInterrupt: 

In [ ]:
fake_imgs_list = train_G_controller_(
    G_cont_params,
    G_controller,
    G_controller_opt,
    gen_net,
    G_prev_hiddens,
    G_prev_archs,
    search_iter,
    writer
)

In [ ]:
train_D_controller_(
        D_cont_params,
        D_controller,
        D_controller_opt,
        dis_net,
        D_prev_hiddens,
        D_prev_archs,
        search_iter,
        writer,
        gen_net
    )

In [ ]:
def count_parameters_in_MB(model):
    return np.sum(np.prod(v.size()) for name, v in model.named_parameters() if "auxiliary" not in name) / 1e6

count_parameters_in_MB(gen_net)

In [ ]:
count_parameters_in_MB(dis_net), count_parameters_in_MB(G_controller)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
a, _, _ = D_controller.sample(1)
a

In [ ]:
dis_net.set_arch([0, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1], 2)
for iter_idx, (imgs, _) in enumerate(train_loader):
    real_imgs = imgs.type(torch.cuda.FloatTensor)
    real_validity = dis_net(real_imgs)
    break

In [ ]:
gen_net.set_arch([0, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 1], 2)
gen_net(torch.cuda.FloatTensor(np.random.normal(0, 1, (1, f_dim)))).shape

In [ ]:
# G output shape:
# stage 0: torch.Size([1, 3, 32, 32])
# stage 1: torch.Size([1, 3, 64, 64])
# stage 3: torch.Size([1, 3, 128, 128])